In [1]:
import os.path

import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
import spacy
from sklearn.preprocessing import LabelEncoder
import os
import torch
from tqdm import tqdm


#from torch_geometric

In [3]:
!python3 -m spacy download en_core_web_lg

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
     |████████████████████████████████| 400.7 MB 7.0 kB/s  eta 0:00:01   |█                               | 12.9 MB 7.9 MB/s eta 0:00:49 MB 62.2 MB/s eta 0:00:06
     |████████████████████████████████| 11.3 MB 12.4 MB/s eta 0:00:01
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.9.0
    Uninstalling pydantic-1.9.0:
      Successfully uninstalled pydantic-1.9.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [5]:
nlp = spacy.load("en_core_web_lg")

In [6]:
df_sample = pd.read_csv('data.csv')

In [7]:
df_sample.head()

,text,source,prompt_id,text_length,word_count
0,"Federal law supersedes state law, and cannabis...",Bloom-7B,0,967,157
1,Miles feels restless after working all day. He...,Bloom-7B,0,5068,778
2,So first of I am danish. That means that I fol...,Bloom-7B,0,1602,267
3,In this paper we present a novel rule-based ap...,Bloom-7B,0,5469,848
4,"Most social progressives, love democracy, and ...",Bloom-7B,0,2379,380


In [163]:
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 788922 entries, 0 to 788921
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   text         788922 non-null  object
 1   source       788922 non-null  object
 2   prompt_id    788922 non-null  int64 
 3   text_length  788922 non-null  int64 
 4   word_count   788922 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 30.1+ MB


In [22]:
df_train=df_sample


In [23]:
len(df_train)

788922

In [24]:
df_train=df_train.drop(['prompt_id','text_length','word_count'], axis=1)

In [25]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 788922 entries, 0 to 788921
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    788922 non-null  object
 1   source  788922 non-null  object
dtypes: object(2)
memory usage: 12.0+ MB


In [26]:
df_train['source'].loc[df_train['source'] != 'Human'] = 'LLM'


In [27]:
df_train.source.value_counts()

LLM      441230
Human    347692
Name: source, dtype: int64

In [28]:
df_train.text.head()

0    Federal law supersedes state law, and cannabis...
1    Miles feels restless after working all day. He...
2    So first of I am danish. That means that I fol...
3    In this paper we present a novel rule-based ap...
4    Most social progressives, love democracy, and ...
Name: text, dtype: object

In [29]:
le = LabelEncoder()
df_train['source']=le.fit_transform(df_train['source'])
print(df_train.source.value_counts())

1    441230
0    347692
Name: source, dtype: int64


In [30]:
df_train.head()

,text,source
0,"Federal law supersedes state law, and cannabis...",1
1,Miles feels restless after working all day. He...,1
2,So first of I am danish. That means that I fol...,1
3,In this paper we present a novel rule-based ap...,1
4,"Most social progressives, love democracy, and ...",1


In [31]:
for i in tqdm(range(len(df_train))):
    df_train.text[i]= df_train.text[i].lower().replace("\n", " ").strip()

  0%|          | 0/788922 [00:00<?, ?it/s]/tmp/ipykernel_5810/1402532788.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.text[i]= df_train.text[i].lower().replace("\n", " ").strip()
100%|██████████| 788922/788922 [1:56:50<00:00, 112.54it/s]  


In [33]:
df_train.text.head()

0    federal law supersedes state law, and cannabis...
1    miles feels restless after working all day. he...
2    so first of i am danish. that means that i fol...
3    in this paper we present a novel rule-based ap...
4    most social progressives, love democracy, and ...
Name: text, dtype: object

In [32]:
print(df_train.source.value_counts())

1    441230
0    347692
Name: source, dtype: int64


In [34]:
df_train.to_csv('train.csv')

In [ ]:
df_train.head()

In [48]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 788922 entries, 156512 to 788921
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    788922 non-null  object
 1   target  788922 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 34.2+ MB


In [ ]:
#df_train=pd.read_csv("train.csv")


In [35]:
class Tokenizer(object):
    def __init__(self, word2idx=None, nlp_model="en_core_web_lg"):
        self.nlp= spacy.load(nlp_model)
        #self.word2idx=word2idx
        #self.idx=len(word2idx)
        if word2idx is None:
            self.word2idx = {}
            self.idx2word = {}
            self.idx = 0
            self.word2idx['<pad>'] = self.idx
            self.idx2word[self.idx] = '<pad>'
            self.idx += 1
            self.word2idx['<unk>'] = self.idx
            self.idx2word[self.idx] = 'unk'
            self.idx += 1
        else:
            self.word2idx = word2idx
            self.idx2word = { v:k for k,v in word2idx.items()}
        
    def fit_on_doc(self, doc:spacy.tokens.doc.Doc):
        for word in doc:
            word= str(word).lower()
            if word not in self.word2idx:
                self.word2idx[word]=self.idx
                self.idx2word[self.idx]=word
                self.idx += 1
    def text_to_doc(self, text):
        return self.nlp(text)
    def doc_to_sequence(self, doc:spacy.tokens.doc.Doc):
        sequence = []
        for w in doc:
            w = str(w).lower()
            word_id = self.word2idx.get(w,-1)
            if word_id == -1:
                word_id = self.word2idx['<unk>']
            sequence.append(word_id)
        if len(sequence) == 0:
            sequence = [0]
        return np.array(sequence, dtype=np.int32)
    def doc_to_adj(self, doc: spacy.tokens.doc.Doc):
        matrix = np.zeros((len(doc),len(doc))).astype('int32')
        for token in doc:
            for child in token.children:
                matrix[token.i][child.i] = 1
                matrix[child.i][token.i] = 1
        return matrix

In [36]:
tokenizer = Tokenizer()

In [37]:
tokenizer.word2idx

{'<pad>': 0, '<unk>': 1}

In [38]:
train_idx2graph = {}
from tqdm import tqdm
for i in tqdm(range(len(df_train))):
    text = df_train.text[i]
    doc = tokenizer.text_to_doc(text)
    tokenizer.fit_on_doc(doc)
    adj_matrix = tokenizer.doc_to_adj(doc)
    coo = coo_matrix(adj_matrix)
    train_idx2graph[i] = np.array([coo.row, coo.col])

100%|██████████| 788922/788922 [23:24:59<00:00,  9.36it/s]     


In [39]:
len(train_idx2graph)

788922

In [40]:
def load_word_vec(path, word2idx=None, embed_dim=300):
    fin = open(path, 'r', encoding='utf8', newline='\n', errors='ignore')
    word_vec = {}
    for line in fin:
        tokens = line.rstrip().split()
        word, vec = ' '.join(tokens[:-embed_dim]), tokens[-embed_dim:]
        if word in word2idx.keys():
            word_vec[word] = np.array(vec, dtype=np.float32)
    return word_vec

In [41]:

def build_embedding_matrix(word2idx, embed_dim=300):
    embedding_matrix = np.zeros((len(word2idx), embed_dim))
    embedding_matrix[1, :] = np.random.uniform(-1/np.sqrt(embed_dim), 1/np.sqrt(embed_dim), (1, embed_dim))

    glob_vector='./glove.840B.300d.txt'
    word_vec = load_word_vec(glob_vector, word2idx=word2idx, embed_dim=embed_dim)

    for word, i in word2idx.items():
        vec = word_vec.get(word)
        if vec is not None:
            embedding_matrix[i] = vec
    return embedding_matrix

In [42]:
embedding_matrix = build_embedding_matrix(tokenizer.word2idx, 300)

In [43]:
print(embedding_matrix)

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.05307571 -0.03220903 -0.05218866 ...  0.02238716  0.02602542
   0.03051941]
 [-0.27522999 -0.38723001  0.31626999 ... -0.37702     0.033019
   0.27742001]
 ...
 [ 0.37158999 -0.51006001 -0.01041    ...  0.85096997 -0.19427
  -0.2895    ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


In [44]:
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${torch.__version__}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${torch.__version__}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.12.1
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


In [45]:
df_train.source.value_counts()

1    441230
0    347692
Name: source, dtype: int64

In [46]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.optim import lr_scheduler
from torch_geometric.data import Data
from torch_geometric.data import InMemoryDataset
from torch_geometric.nn import GCNConv, GATv2Conv 
import torch_geometric.nn as pyg_nn
from torch_geometric.loader import DataLoader

In [47]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [48]:

class TrainGraphFactoryDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)

        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_dir(self):
        return "./"

    @property
    def processed_dir(self):
        return os.path.join(self.root, "train_processed")

    @property
    def raw_file_names(self):
        return ['train.csv']

    @property
    def processed_file_names(self):
        return ['train-graph.pt']

    def download(self):
        pass

    def process(self):

        data_list = self.read_data()

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

    def read_data(self):
        global df_train
        df_train_data = df_train
        all_data = []
        for i in tqdm(range(df_train_data.shape[0])):
            text = df_train_data.text[i]
            doc = tokenizer.text_to_doc(text)
            input_ids = tokenizer.doc_to_sequence(doc)
            label = df_train_data.loc[i, ["source"]].to_list()

            x = torch.tensor(input_ids.reshape((-1, 1)), dtype=torch.int32)
            edge_index  = torch.tensor(train_idx2graph[i], dtype=torch.long)
            y = torch.tensor(np.array(label), dtype=torch.float32).reshape(-1, 1)
            data = Data(x=x, edge_index=edge_index, y=y)

            all_data.append(data)
        return all_data

In [49]:
train_dataset = TrainGraphFactoryDataset(root='./')

Processing...
100%|██████████| 788922/788922 [21:53:54<00:00, 10.01it/s]    
Done!


In [50]:
print(train_dataset.raw_paths, train_dataset.processed_paths)

['./train.csv'] ['./train_processed/train-graph.pt']


In [51]:
train_dataset[2]

Data(x=[315, 1], edge_index=[2, 602], y=[1, 1])

In [52]:
train_dataset[20000].y

tensor([[1.]])

In [53]:
len(train_dataset)

788922

In [54]:
dataset = train_dataset.shuffle()
ratio_cut= 0.8
ratio_valid= 0.1
train_len = int(ratio_cut*len(dataset))
valid_len = len(dataset)-int(ratio_valid*len(dataset))
train_dataset= dataset[:train_len]
val_dataset= dataset[train_len:valid_len]
test_dataset = dataset[valid_len:]
print(len(train_dataset),len(val_dataset), len(test_dataset))


631137 78893 78892


In [ ]:
from torch_geometric.data import DataLoader

train_loader = DataLoader(train_dataset,batch_size=32)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:
print(embedding_matrix)

In [ ]:
valid = []
invalid = []
for data in tqdm(train_loader):
    try:
        if data.validate():
            valid.append(data)
        else:
            invalid.append(i)
    except ValueError as ie:
        print('error in train edge index')


val_valid = []
val_invalid = []
for  data in tqdm(val_loader):
    try:
        if data.validate():
            val_valid.append(data)
        else:
            val_invalid.append(i)
    except IndexError as ie:
        print('error in val edge index')

In [ ]:
(len(valid)/len(train_loader))*100

In [ ]:
(len(invalid)/len(train_loader))*100

In [ ]:
(len(val_valid)/len(val_loader))*100

In [ ]:
(len(val_invalid)/len(val_loader))*100

In [ ]:
from torch_geometric.data import Batch

class FeedbackModel(nn.Module):
    def __init__(self, embedding_matrix):

        super(FeedbackModel, self).__init__()

        self.embed = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float), freeze=False)
        # GCNConv SAGEConv ResGatedGraphConv GraphConv(300, 128) 
        # TransformerConv GATv2Conv GATConv(300, 128, heads=4) ChebConv(300, 128, K=2)
        # GCNConv SAGEConv ResGatedGraphConv GraphConv(128, 64) 
        # TransformerConv  GATv2Conv GATConv(4*128, 64) ChebConv(128, 64, K=2)
        #         self.gru = nn.GRU(256, 256, num_layers=1, 
        #                           dropout=0, batch_first=True,
        #                           bidirectional=False)          # RNN, GRU
        # output: (N, L, D∗Hout), D = 2 if bidirectional=True otherwise 1
        # h_n: (D∗num_layers, N, Hout)
        self.gc1   = GATv2Conv(300, 128, heads=4)
        #self.gc1   = GraphSAGE (300,128, 32)
        self.pool1 = pyg_nn.TopKPooling(128, ratio=0.8)
        self.gc2   = GCNConv(128, 64)
        self.pool2 = pyg_nn.TopKPooling(64, ratio=0.8)
        self.lin1  = nn.Linear(64,32)
        self.lin2  = nn.Linear(32, 1)

    def forward(self, data):
        data_list = [Data(x=x_, edge_index=data.edge_index, sample_batched=data.batch) for x_ in (data.x)]
        batch = Batch.from_data_list(data_list)
        print('(Data List Loaded)')
        x = batch.x
        sample_batch=batch.sample_batched
        #x.squeeze(1)
        #print('(squeeze passed)')
        x = torch.clamp(x, 0, self.embed.num_embeddings - 1)
        print('(embed 1 pre-Loaded')
        x = self.embed(x)
    
        print('(Layer 1 pre-Loaded)')
        x = F.relu(self.gc1(x, edge_index=batch.edge_index))
        print('(Layer 1 Loaded)')
        x, edge_index, edge_attr, batch2, perm, score = self.pool1(x, batch.edge_index, None, sample_batch)
        x1 = torch.cat([pyg_nn.global_max_pool(x, batch2), pyg_nn.global_mean_pool(x, batch2)], dim=1)

        x = F.relu(self.gc2(x, batch.edge_index))
        print('(Layer 2 Loaded)')
        x, edge_index, edge_attr, batch2, perm, score = self.pool2(x, data.edge_index, None, sample_batch)
        x2 = torch.cat([pyg_nn.global_max_pool(x, batch2), pyg_nn.global_mean_pool(x, batch2)], dim=1)

        x = x1 + x2
        output = F.relu(self.lin1(x))
        print('(Layer 3 Loaded)')
        x = F.dropout(x, p=0.5, training=self.training)
        output = F.relu(self.lin2(x))
        print('(outputs loaded)')
        return output



In [ ]:
model = FeedbackModel(embedding_matrix)
model

In [ ]:
epochs = 20

criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=1e-5)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=6)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion.to(device)

In [ ]:



total_loss = []
for epoch_num in range(epochs):

    model.train()
    total_loss_train = 0
    print(f'epoch: {epoch_num} in progress...')
    print(f'train computing')
    
    for sample_batched in tqdm(valid):
        print('in train loop')
        sample_batched = sample_batched.to(device)
        optimizer.zero_grad()
        outputs = model(sample_batched)
        print('outputs loaded')
        label = sample_batched.y.to(device)
        loss = criterion(outputs, label)
        loss.backward()
        total_loss_train += loss.item()
        optimizer.step()
        

    model.eval()
    total_loss_val = 0
    
    print(f'train evaluating')
    with torch.no_grad():
        for sample_batched in tqdm(val_valid):
                sample_batched = sample_batched.to(device)
                outputs = model(sample_batched)
                label = sample_batched.y.to(device)
                loss = criterion(outputs, label)
                total_loss_val += loss.item()

    scheduler.step(total_loss_val / len(val_dataset))

    print(f'Epoch: %02.0f ended | Train Loss: {total_loss_train / len(train_dataset): .3f} | Val Loss: {total_loss_val / len(val_dataset): .3f}' % (epoch_num + 1))
    total_loss.append([total_loss_train / len(train_dataset), total_loss_val / len(val_dataset)])

